In [52]:
import gkeepapi
from gkeepapi import has_attr_value, get_task_id, convert_to_task, add_to_taskwarrior,  perform_incremental_update
from pprint import pprint as print
keep = gkeepapi.Keep()
success = keep.login('achalagarwal.01@gmail.com', 'kvlatubhkxhlldsi')

In [51]:
import subprocess
import re
from logging import Logger

In [3]:
# proc = subprocess.Popen(['task', 'add', 'test_child' ], stderr=subprocess.PIPE, stdout=subprocess.PIPE)
# stdout_value = proc.communicate()
# print('\tstdout:' + str(stdout_value))

In [4]:
#string = stdout_value[0].decode('ascii')

In [5]:
#re.findall(r'\d+', string )

In [6]:
# proc = subprocess.Popen(['task', '23', 'delete' ], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE)
# # proc.stdin()
# yes = bytes('yes \n', 'ascii')
# stdout_value = proc.communicate(yes)
# # stdout_value = proc.communicate(b'all \n')
# print('\tstdout:' + str(stdout_value))

In [7]:
# string = stdout_value[0].decode('ascii')

In [25]:
# re.findall(r'Deleted [1-9]+ task.', string) 

In [32]:
def convert_to_task(node):

    # create base structure
    task = {'text': node.text,
            'keep_id':node.id,
            'children': node._subitems,
            'created':node.timestamps._created,
            
            # only records parents for sublist items
            # as a guide, this should be None anytime its parent is irrelevant to Task Warrior

            'parent': node.parent_id if type(node.parent_item) is gkeepapi._node.ListItem else None
            }
    
    # populate with children
    task['children'] = list(map(
                            lambda x: convert_to_task(x), 
                            filter(lambda x: has_attr_value(x, 'checked', False),
                            task.get('children', []).values())))

    return task


In [9]:
def add_to_taskwarrior(task, parent=None, project='Google Keep'):

    # add task in taskwarrior
    proc = subprocess.Popen(['task', 'add', task['text'] ], stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout_value = proc.communicate()

    # extract taskwarrior id from stdout
    string = stdout_value[0].decode('ascii')
    task_id = re.findall(r'\d+', string )[0]
    task['task_id'] = task_id

    # now we annotate the google keep task id
    subprocess.Popen(['task', task['task_id'], 'annotate', task['keep_id']]).communicate()

    # add as a dependent to parent
    if parent:
        proc = subprocess.Popen(['task', parent, 'modify', 'depends:'+task['task_id'] ], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE)
        stdout_value = proc.communicate()

    # iterate for all children
    for child in task['children']:
        add_to_taskwarrior(child, parent=task['task_id'], project=project)


In [10]:
def delete_from_taskwarrior(tasks):

    if not tasks:
        return True
    # create arguments depending on whether tasks is a single number or a list of numbers
    # TODO Should we enforce string input here?
    task_string, confirm_string, count = (','.join([str(task) if task else '' for task in tasks]), 'all \n', len(tasks)) if hasattr(tasks, '__iter__') else (str(tasks), 'yes \n', 1)

    # delete tasks from taskwarrior
    proc = subprocess.Popen(['task', task_string, 'delete' ], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE)

    # confirm deletion of all tasks provided
    stdout_value = proc.communicate(bytes(confirm_string, 'ascii'))
    string = stdout_value[0].decode('ascii')
    return True if re.findall(rf'Deleted {count}+ task.', string) else False
        


In [11]:
# proc = subprocess.Popen(['task', '25', 'modify', 'depends:26' ], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE)
# # proc.stdin()
# stdout_value = proc.communicate()

# # stdout_value = proc.communicate(b'yes \n')

# print('\tstdout:' + str(stdout_value))

In [12]:
def has_attr_value(obj, attr='', val=''):
    return getattr(obj, attr) == val

In [13]:
notes = keep.all()

In [14]:
filtered_notes = list(filter(lambda x: has_attr_value(x, 'pinned', True), notes))

In [15]:
filtered_notes

In [16]:
filtered_notes = list(filter(lambda x: x.title, filtered_notes))

In [17]:
filtered_notes

In [18]:
for note in filtered_notes:
    print(note.id + note.title)

'16889e295a0.a03e63d51aa58e19Significant + Urgent'
'16889e3e4c7.938359362aa74d01Significant'
'16889e48842.80bf82edf6e55e71Urgent'
'1510551568605.2096093560Today'
'1702b89474c.a5e79116461ef293Shopping List'
'1584490858625.2086780820Not Significant + Not Urgent'
'1582912357250.1423760568Memory Bank for Reflect + Recollect'


In [19]:
note = list(filter(
    lambda x: has_attr_value(x, 'title', 'Significant + Urgent'), 
    notes))[0]
items = note.items

In [20]:
# print(dir(items[0]))

In [35]:
# if parent has pinned and is :)
# .__dict__
list(items[12]._subitems.values())[1].__dict__

{'_dirty': False,
 'parent': <gkeepapi.node.List at 0x7f5250b89cc0>,
 'id': '1584141206679.1825644213',
 'server_id': '17DU3uvReMk63f5RROxYcq87hmniHwo7FnsRkUPhIhfWX1bEGFb3NjMQAMNoX2NI',
 'parent_id': '16889e295a0.a03e63d51aa58e19',
 'type': <NodeType.ListItem: 'LIST_ITEM'>,
 '_sort': '692355879230853',
 '_version': '1',
 '_text': 'Check drafts',
 '_children': {},
 'timestamps': <gkeepapi.node.NodeTimestamps at 0x7f52502fa828>,
 'settings': <gkeepapi.node.NodeSettings at 0x7f52502fa8d0>,
 'annotations': <gkeepapi.node.NodeAnnotations at 0x7f52502fa898>,
 '_moved': False,
 'parent_item': <gkeepapi.node.ListItem at 0x7f5247f99d68>,
 'parent_server_id': None,
 'super_list_item_id': '16feb45ae35.ab86dba36e485297',
 'prev_super_list_item_id': None,
 '_subitems': {},
 '_checked': True}

In [48]:
# remove tasks which are already completed
filtered_items = list(filter(lambda x: has_attr_value(x, 'checked', False), items))

In [49]:
[item.text for item in filtered_items]

['Read your algorithm, read other algorithms',
 'set deadlines',
 'I know what I want to do, I want to be able to ask myself what type of work I want to do and then go ahead and get the "next" best task to do!, task warrior really helps with that and this is why I want the task warrior integration! ',
 'Update google form for reflections',
 'Callbacks and review of what you wrote previously \n- 3 to 5 day goals can be shown to the person\n- list of tasks that were meant to be completed could be shown?',
 'what other things would you like to track statistically? I am generating data here, high quality data, mention your recreation, mention what you studied, etc mention these few things in the description box. in bath section, ',
 'affirmation should be something that I should chant while meditation, extract that out from sheets and send it as a notification/email',
 'clean mantras']

In [23]:
# remove tasks which are children of some other task
filtered_items = list(filter(lambda x: has_attr_value(x, 'parent_item', None ), filtered_items))

In [33]:
filtered_tasks = [convert_to_task(x) for x in filtered_items]

In [34]:
filtered_tasks

[{'text': 'Read your algorithm, read other algorithms',
  'keep_id': '1584344571475.1329806704',
  'children': [],
  'created': datetime.datetime(2020, 3, 16, 7, 42, 52, 54000),
  'parent': None},
 {'text': 'set deadlines',
  'keep_id': '1584951671107.1572702742',
  'children': [],
  'created': datetime.datetime(2020, 3, 23, 8, 21, 11, 536000),
  'parent': None},
 {'text': 'I know what I want to do, I want to be able to ask myself what type of work I want to do and then go ahead and get the "next" best task to do!, task warrior really helps with that and this is why I want the task warrior integration! ',
  'keep_id': '1584141228696.1680011852',
  'children': [],
  'created': datetime.datetime(2020, 3, 13, 23, 13, 50, 109000),
  'parent': None},
 {'text': 'Update google form for reflections',
  'keep_id': '170b2abcb55.a64cca6983f69347',
  'children': [{'text': 'Callbacks and review of what you wrote previously \n- 3 to 5 day goals can be shown to the person\n- list of tasks that were m

In [42]:
# filtered_tasks

In [26]:
# for item in filtered_tasks:
#     children = item.get('children').values()
#     item['children'] = list(map(lambda x: {'text': x.text, 'keep_id': x.id, 'children':[], 'created': x.timestamps._created},filter(lambda x: has_attr_value(x, 'checked', False), children)))

In [27]:
filtered_tasks[0]

{'text': 'Read your algorithm, read other algorithms',
 'keep_id': '1584344571475.1329806704',
 'children': [],
 'created': datetime.datetime(2020, 3, 16, 7, 42, 52, 54000)}

In [28]:
filtered_items[0].timestamps.__dict__

{'_dirty': False,
 '_created': datetime.datetime(2020, 3, 16, 7, 42, 52, 54000),
 '_deleted': None,
 '_trashed': None,
 '_updated': datetime.datetime(2020, 3, 23, 8, 20, 24, 369000),
 '_edited': None}

In [29]:
filtered_items[4].__dict__

IndexError: list index out of range

In [24]:
# so we have the tasks!
# now we have to exec the task add command get the task id, use that to link children!

In [25]:
# for loop
# add task_warrior id in dict
# need to get that by parsing the subprocess.PIPE.etc

In [30]:
for task in filtered_tasks:
    print(task)

{'children': [],
 'created': datetime.datetime(2020, 3, 16, 7, 42, 52, 54000),
 'keep_id': '1584344571475.1329806704',
 'text': 'Read your algorithm, read other algorithms'}
{'children': [],
 'created': datetime.datetime(2020, 3, 23, 8, 21, 11, 536000),
 'keep_id': '1584951671107.1572702742',
 'text': 'set deadlines'}
{'children': [],
 'created': datetime.datetime(2020, 3, 13, 23, 13, 50, 109000),
 'keep_id': '1584141228696.1680011852',
 'text': 'I know what I want to do, I want to be able to ask myself what type '
         'of work I want to do and then go ahead and get the "next" best task '
         'to do!, task warrior really helps with that and this is why I want '
         'the task warrior integration! '}
{'children': [{'children': [],
               'created': datetime.datetime(2020, 2, 27, 8, 13, 47, 267000),
               'keep_id': '1582791225563.1845751236',
               'text': 'Callbacks and review of what you wrote previously \n'
                       '- 3 to 5 day 

In [35]:
# keep._keep_api.changes(keep._keep_version)
keep._keep_api.changes(keep._keep_version).get('nodes')

In [33]:
keep.sync(True)

In [56]:
# 

In [69]:
# now I need to perform incremental updates on my taskwarrior entries
# so for that I need to be able to extract task id from annotations!

# the incremental updates are limited to checked/unchecked and/or adding children, because they lead to changes in parents, however will it be tricky to get parents and children updates when we need the task id?

# i dont know this right now because of obvious reasons
# firstly i dont know in what form the incremental updates arrive 
# if they are of the form, a child has been added, heres the new child, 
# dont have to use a dictionary for efficiency right now as we have enough compute power






In [ ]:
def perform_incremental_update(nodes, current):
    # nodes are the latest updates that have been fetched from the api
    # current is the current set of filtered tasks
    # 
    proc = subprocess.Popen(['task','aws' ], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE)
    stdout_value = proc.communicate()

    # extract task id from stdout output 
    string = stdout_value[0].decode('ascii')
    task_id = re.findall('\d+', string)[0]
    # an alternate way would be to split the lines
    # task_details = string.split('\n')[3]

In [110]:
def get_task_id(keep_id):
    proc = subprocess.Popen(['task',keep_id ], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE)
    stdout_value = proc.communicate()

    # extract task id from stdout output 
    string = stdout_value[0].decode('ascii')
    return re.findall('\d+', string)[0] if string else None
    # an alternate way would be to split the lines
    # task_details = string.split('\n')[3]

In [101]:
proc = subprocess.Popen(['task','please' ], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE)
stdout_value = proc.communicate()

# extract task id from stdout output 
string = stdout_value[0].decode('ascii')
if string:
    task_id = re.findall('\d+', string)[0]
else:
    task_id = None
# an alternate way would be to split the lines
# task_details = string.split('\n')[3]

In [113]:
# ASSERT that I will not be changing parents
# only adding entries or subentries
# so now hook onto the create_list, update_list
# and also hook on the the delete_list
# because I dont care about anything else